In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('recommender').getOrCreate()
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

In [ ]:
# SIMPLE WAY

In [2]:
import pandas as pd
reviews = pd.read_csv('data/reviews/all_reviews.csv')
datatest, datatrain  = reviews.sample(frac =.2), reviews.sample(frac=.8)

In [3]:
reviews.shape

(4939, 6)

In [4]:
DataTrain = spark.createDataFrame(datatrain)
DataTest = spark.createDataFrame(datatest)

In [5]:
als = ALS(userCol='userId', itemCol='itemId', ratingCol='rating', nonnegative=True)

model = als.fit(DataTrain)

# Generate Predictions and evaluate using RMSE
predictions = model.transform(DataTest)
predictions = predictions.na.drop()

In [6]:
predictions.describe().show()

+-------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+
|summary|        Unnamed: 0|          reviewId|            itemId|            userId|           testimony|            rating|        prediction|
+-------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+
|  count|               855|               855|               855|               855|                 855|               855|               855|
|   mean|2355.4795321637425|2604.7380116959066|12.508771929824562|1866.5345029239766|                null| 4.543859649122807|  4.34432890171196|
| stddev|1388.1246524754483|1432.6355755759657|  8.29432837247834|1100.0060819142936|                null|0.8184818178392029|0.7961369056323515|
|    min|                 0|                 0|                 1|                 2|(accidentally pos...|                 1|     

In [7]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', 
                                predictionCol="prediction")    
# RMSE Root Mean Square Error
rmse = evaluator.evaluate(predictions)

print("RMSE = " + str(rmse))

RMSE = 0.3688342148417594


In [ ]:
# from pyspark.sql.functions import lit

# def recommendMovies(model, user, nbRecommendations):
#     # Create a Spark DataFrame with the specified user and all the movies listed in the ratings DataFrame
#     dataSet = ratings.select("movieId").distinct().withColumn("userId", lit(user))

#     # Create a Spark DataFrame with the movies that have already been rated by this user
#     moviesAlreadyRated = ratings.filter(ratings.userId == user).select("movieId", "userId")

#     # Apply the recommender system to the data set without the already rated movies to predict ratings
#     predictions = model.transform(dataSet.subtract(moviesAlreadyRated)).dropna().orderBy("prediction", ascending=False).limit(nbRecommendations).select("movieId", "prediction")

#     # Join with the movies DataFrame to get the movies titles and genres
#     recommendations = predictions.join(movies, predictions.movieId == movies.movieId).select(predictions.movieId, movies.title, movies.genres, predictions.prediction)

#     recommendations.show(truncate=False)
